# Workflow test for `micaflow`

## Inputs in BIDS

- BIDS directory example: `/data/mica3/BIDS_MICs/rawdata`

- subject for test `sub-HC126`

- session `ses-01`

- T1w input test: `/data/mica3/BIDS_MICs/rawdata/sub-HC126/ses-01/anat/sub-HC126_ses-01_T1w.nii.gz`


## Output structure
where type can be `T1w`, `FLAIR`, or `T2w`.
https://bids-specification.readthedocs.io/en/stable/modality-specific-files/magnetic-resonance-imaging-data.html


BIDS specifications. 
`OUTDIR/sub-SUBJECT/ses-SESSION/anat/sub-SUBJECT_ses-SESSION_TYPE.nii.gz`


## Dependencies
1. freesurfer 7.4.0

2. ANTS maybe yes maybe no

3. python. What packages?

## workflow 
1. Bias field correction (ANTs) 

2. Brain mask `mri_synthstrip` Maybe this step can be avoided and we might be able to use the brain segmentation
3. Normalization

4. Brain segmentation `synth_seg` from freesurfer
5. Registration to MNI


### Intensity Non-uniform correction - N4
```bash
N4BiasFieldCorrection  -d 3 -i "$T1n4" -r -o "$T1n4" -v
```

### Brain mask
```bash
mri_synthstrip -i "$T1nativepro" -o "$T1nativepro_brain" -m "$T1nativepro_mask" --no-csf
```

### Normalization `02_proc_flair`

### Segmentation 'synth_seg'

```bash
# Define variables
out=/host/yeatman/local_raid/rcruces/data/tmp

img_fixed=/data_/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC019/ses-03/anat/sub-PNC019_ses-03_space-nativepro_T1w_nlm.nii.gz

img_moving=/data_/mica3/BIDS_PNI/derivatives/Denoised_T2star/10PNC_nomask/sub-PNC019_ses-03_B1-corrected_T2starmap_nomask_denoised.nii

id=sub-PNC019_ses-03
outname=${out}/${id}_space-nativepro_T2star.nii.gz

threads=15

# Use synthseg to get the labels of the fixed and moving image
mri_synthseg --i ${img_moving} \
             --o ${out}/${id}_moving_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

mri_synthseg --i ${img_fixed} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

```

### Registration to MNI `easy_reg`
```bash
# Calculate the warpfield and affine transformation with easyreg
mri_easyreg --ref ${img_fixed} \
            --flo ${img_moving} \
            --ref_seg ${out}/${id}_fixed_synthseg.nii.gz \
            --flo_seg ${out}/${id}_moving_synthseg.nii.gz \
            --fwd_field ${out}/${id}_from-moving_to-fixed_mode-image_desc-easyreg.nii.gz \
            --threads ${threads} 
```

In [ ]:
%%bash
out=/host/yeatman/local_raid/cerys/data

img_fixed=/host/yeatman/local_raid/cerys/Downloads/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a.nii

input=/data/mica3/BIDS_MICs/rawdata/sub-HC126/ses-01/anat/sub-HC126_ses-01_T1w.nii.gz

## T1n4, T1nativepro, T1nativepro_brain, T1nativepro_mask?

id=sub-HC126_ses-01

atlas= mri_synthseg --i ${img_fixed} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

In [ ]:
## 01_proc-structural.sh variables
T1str_nat="${idBIDS}_space-nativepro_T1w"
T1reo="${tmp}/${T1str_nat}_reo.nii.gz"
T1n4="${tmp}/${T1str_nat}_n4.nii.gz"
T1nativepro="${proc_struct}/${T1str_nat}.nii.gz"
T1nativepro_brain="${proc_struct}/${idBIDS}_space-nativepro_T1w_brain.nii.gz"
T1nativepro_mask="${proc_struct}/${idBIDS}_space-nativepro_T1w_brain_mask.nii.gz"

In [ ]:
%%bash
# Bias field correction (ANTS)
N4BiasFieldCorrection  -d 3 -i input -r -o [ corrected_T1w.nii.gz, T1w_BiasField.nii.gz ] -v

In [ ]:
%%bash
## 01_proc-structural.sh code

Info "T1w_nativepro biasfield correction and intensity rescaling"
# Intensity Non-uniform correction - N4
Do_cmd N4BiasFieldCorrection  -d 3 -i "$T1n4" -r -o "$T1n4" -v

# Rescale intensity [100,0]
Do_cmd ImageMath 3 "$T1nativepro" RescaleImage "$T1n4" 0 100

# If no T1native pro exit_status "something is wrong" exit
if [ ! -f "$T1nativepro" ]; then Error "$T1str_nat was not generated"; Do_cmd exit; fi

# Brainmask
Do_cmd mri_synthstrip -i "$T1nativepro" -o "$T1nativepro_brain" -m "$T1nativepro_mask" --no-csf

# If no T1native pro exit_status "something is wrong" exit
if [ ! -f "$T1nativepro_brain" ]; then Error "$T1str_nat masked was not generated"; Do_cmd exit; else ((Nsteps++)); fi

# Create json file for T1native
json_nativepro_T1w "$T1nativepro" "$Nimgs" "${bids_T1ws[*]}" "${procstruct_json}"

In [ ]:
%%bash
# Brain mask
mri_synthstrip -i "$T1nativepro" -o "$T1nativepro_brain" -m "$T1nativepro_mask" --no-csf

In [ ]:
%%bash
# Normalization
antsRegistrationSyNQuick.sh -d 3 -f "$T1n4" -m "$img_fixed" -o "$T1n4" -t s -j 1

In [ ]:
%%bash
# Segmentation
threads=15

mri_synthseg --i ${img_moving} \
             --o ${out}/${id}_moving_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

mri_synthseg --i ${img_fixed} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

In [ ]:
%%bash
# Registration to MNI
mri_easyreg --ref ${img_fixed} \
            --flo ${img_moving} \
            --ref_seg ${out}/${id}_fixed_synthseg.nii.gz \
            --flo_seg ${out}/${id}_moving_synthseg.nii.gz \
            --fwd_field ${out}/${id}_from-moving_to-fixed_mode-image_desc-easyreg.nii.gz \
            --threads ${threads}